In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Regular EDA(exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

from tqdm import tqdm
import string
import statistics
import re
import joblib
import random
import statistics
from collections import Counter
from scipy import stats
import os
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
from matplotlib.lines import Line2D

import tensorflow as tf
import torch

## Some findings

In [3]:
import warnings

# Supress the specific warning about file system plugins not being loaded
warnings.filterwarnings("ignore")

In [4]:
from transformers import BertModel, BertTokenizer, BertConfig, AutoTokenizer

from transformers import AutoModelForPreTraining, AutoTokenizer


# # from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
# import torch

# model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert_large",  output_hidden_states = True)
# tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large")


In [5]:
# Initialize Bangla Bert From BUET NLP
model = BertModel.from_pretrained('SarwarShafee/BanglaBert_with_TFModel',
           output_hidden_states = True)


tokenizer = BertTokenizer.from_pretrained('SarwarShafee/BanglaBert_with_TFModel')

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertModel were not initialized from the model checkpoint at SarwarShafee/BanglaBert_with_TFModel and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [6]:
def bert_text_preparation(text, tokenizer):
    """
    Preprocesses text input in a way that BERT can interpret.
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)
    
    # Convert inputs to tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segment_tensor = torch.tensor([segments_ids])
    
    return tokenized_text, tokens_tensor, segment_tensor

In [7]:
def get_bert_embeddings(tokens_tensor, segments_tensor, model):
    '''
    Obtains BERT embeddings for tokens, in context of the given sentence.
    '''
    
    # gradient calculation id disabled
    with torch.no_grad():
        # obtain hidden states
        outputs = model(tokens_tensor, segments_tensor)
        hidden_states = outputs[2]

    # concatenate the tensors for all layers
    # use "stack" to create new dimension in tensor
    token_embeddings = torch.stack(hidden_states, dim=0)

    # remove dimension 1, the "batches"
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # swap dimensions 0 and 1 so we can loop over tokens
    token_embeddings = token_embeddings.permute(1,0,2)

    # intialized list to store embeddings
    token_vecs_sum = []

    # "token_embeddings" is a [Y x 12 x 768] tensor
    # where Y is the number of tokens in the sentence

    # loop over tokens in sentence
    for token in token_embeddings:

        # "token" is a [12 x 768] tensor

        # sum the vectors from the last four layers
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)

    return token_vecs_sum

In [8]:
sentence = 'বাংলাদেশের অবস্থা এখন একদমই ভালো না।'
tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(sentence, 
                                                                            tokenizer)
print(tokenizer.tokenize(sentence))
print(tokenized_text)
print(tokens_tensor)
print(segments_tensors)
vect = get_bert_embeddings(tokens_tensor, segments_tensors, model)
print(vect[0])

['বাংলাদেশের', 'অবস্থা', 'এখন', 'একদমই', 'ভালো', 'না', '।']
['[CLS]', 'বাংলাদেশের', 'অবস্থা', 'এখন', 'একদমই', 'ভালো', 'না', '।', '[SEP]']
tensor([[    2,  1631,  2234,  1004, 13995,  1055,   795,   205,     3]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([ 2.7523e-03, -1.1081e-01,  1.1971e+00,  4.4322e-04, -1.9124e-01,
        -2.6116e-01, -1.8375e-01,  4.1139e-01, -3.7679e-01, -1.8421e-01,
         5.1811e-01,  6.8532e-01,  8.5532e-03,  3.5182e-01, -2.6650e-01,
         6.0930e-01, -5.5053e-01, -2.3644e-01,  1.9100e-02, -1.0601e-01,
        -4.5562e-01,  9.4542e-02, -1.8104e-02,  2.5044e-01, -5.5986e-01,
         2.0537e-01,  3.0988e-01,  3.8797e-01, -3.8270e-01, -7.0436e-01,
         3.3084e-01,  4.0949e-01, -7.6140e-01, -2.5828e-01, -8.4948e-02,
         5.6572e-02,  2.0891e-01,  3.8856e-01, -4.5029e-01,  3.1907e-01,
        -1.7881e-01,  6.2828e-01,  1.5136e-01, -1.1826e-01,  3.1038e-01,
         3.5826e-01, -3.7104e-01, -5.2830e-01, -2.0803e+01,  5.7077e-01,
         2.0667e-01,

In [9]:
sentence = 'পদ্মা একটি নদীর নাম। আমি দীর্ঘ নিঃশ্বাস ফেলে বললাম, জ্বি আচ্ছা বদলাব'
tokenizer.tokenize(sentence)

['পদ্মা',
 'একটি',
 'নদীর',
 'নাম',
 '।',
 'আমি',
 'দীর্ঘ',
 'নিঃশ্বাস',
 'ফেলে',
 'বললাম',
 ',',
 'জ্বি',
 'আচ্ছা',
 'বদলা',
 '##ব']

In [10]:
# Necessary Functions

def read_stories(paths):
    '''
    Read All the stories from a folder and return an array of sentence.
    '''
    sents = []
    for path in paths:
        # File path with Bengali characters
        file_path_bangla = path
        # Open the file using triple quotes around the file path
        with open(f"""{file_path_bangla}""", 'r', encoding='utf-8') as file:
            content = file.read()
        sent = make_sentences(content)
        sents.append(sent)
    # Convert the 2d array into 1d array
#     ments = [element for row in sents for element in row]
    return sents


def remove_noise(sentence):
    '''
    Removes noise from the dataset
    '''
    # Remove punctuation
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    # Remove extra spaces and newlines
    sentence = ' '.join(sentence.split())
    return sentence

def make_sentences(content):
    '''
    Takes a full story of a book or Multiple Books
    And convert it into a list of sentences.
    '''
    import re

    # Split the story into sentences using full stops (".") as delimiters
    # sentences = story.strip().split('।')
    sentences = re.split(r'[।?!]', content)

    # Remove any leading/trailing whitespaces from each sentence, remove punctuation and
    # other noise

    # Filter out empty sentences (if any) using the filter function
    sentences = list(filter(None, sentences))
    cleaned_sentences = [remove_noise(sentence) for sentence in sentences if sentence.strip()]

    # Print the list of cleaned sentences
    return cleaned_sentences


## Take Out All the Paths from input directory

In [11]:
paths = []
for dirname, _, filenames in os.walk('/kaggle/input/bookscollection/River Novels 3'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
paths[:4]

['/kaggle/input/bookscollection/River Novels 3/1. Padma.txt',
 '/kaggle/input/bookscollection/River Novels 3/3. Isamoti.txt',
 '/kaggle/input/bookscollection/River Novels 3/1. Padma(1).txt',
 '/kaggle/input/bookscollection/River Novels 3/2. Titas.txt']

In [12]:
def get_book_list(path):
    paths = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            filename = filename.replace('.txt', '')
            paths.append(filename)
            
    return paths

In [13]:
def create_sentences(path):
    '''
    Chill
    '''
    paths = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            paths.append(os.path.join(dirname, filename))

    sentences = read_stories(paths)
    return sentences

In [14]:
sentences = create_sentences('/kaggle/input/bookscollection/River Novels 3')

In [15]:
len(sentences[0])

3501

In [16]:
sentences[0][3490:]

['ফিরা আবার জেল খাটাইব',
 '’ কপিলা আর কথা বলে না',
 'ঘাটের আর খানিক তফাতে হোসেনের প্রকাণ্ড নৌকাটি নোঙর করা ছিল',
 'একজন মাঝি ঘুমাইয়া ছিল নৌকায়',
 'তাহাকে ডাকিয়া তুলিলে সে নৌকা তীরে ভিড়াইল',
 'কুবের নীরবে নৌকায় উঠিয়া গেল',
 'সঙ্গে গেল কপিলা',
 'ছইএর মধ্যে গিয়া সে বসিল',
 'কুবেরকে ডাকিয়া সে বলিল ‘আমারে নিবা মাঝি লগে',
 '’ হ কপিলা চলুক সঙ্গে',
 'একা অতদূরে কুবের পাড়ি দিতে পারব না']

In [17]:
rand_index = np.random.randint(0, len(sentences[0]))
print(f'Index = {rand_index},  sentence = {sentences[0][rand_index]}')

Index = 1930,  sentence = ’ বুঝিতে পারা যায় কপিলা কাঁদিতেছে


In [18]:
from collections import OrderedDict
from tqdm import tqdm

def get_token_vector_list(sentences):
    '''
    Takes an array of sentences and return the individual words(token) 
    And their corresponding Tensor.
    '''
    context_embeddings = []
    context_tokens = []

    for sentence in tqdm(sentences):
        tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(sentence, 
                                                                                tokenizer)
        list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

        # make ordered dictionary to keep track of the position of each word
        tokens = OrderedDict()

        # loop over tokens in sensitive sentence
        for token in tokenized_text[1:-1]:
            # keep track of position of word and whether it occurs multiple times
            if token =='[UNK]':
                continue
            else:
                if token in tokens:
                    tokens[token] += 1
                else:
                    tokens[token] = 1

                # compute the position of the current token
                token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
                current_index = token_indices[tokens[token]-1]

                # get the corresponding embedding
                token_vec = list_token_embeddings[current_index]

                # save values
                context_tokens.append(token)
                context_embeddings.append(token_vec)
                
    return context_tokens, context_embeddings

In [19]:
li_book = get_book_list('/kaggle/input/bookscollection/River Novels 3')

In [20]:
print(f'Token and Embedding for {li_book[0]} and {li_book[1]}')
context_token, context_embedding = get_token_vector_list(sentences[0])
context_token2, context_embedding2 = get_token_vector_list(sentences[1])

Token and Embedding for 1. Padma and 3. Isamoti


100%|██████████| 10196/10196 [11:39<00:00, 14.57it/s]


In [21]:
# print(content)

In [33]:
print(len(context_token2))
context_token2[:20]

103191


['ইছ',
 '##ামত',
 '##ী',
 'একটি',
 'ছোট',
 'নদী',
 'অন্তত',
 'যশোর',
 'জেলার',
 'মধ্য',
 'দিয়ে',
 'এর',
 'যে',
 'অংশ',
 'প্রবাহিত',
 'সেট',
 '##ুকু',
 'দক্ষিণে',
 'ইছ',
 '##ামত']

## Function to get the cosine similarity

In [34]:
context_embedding[0].shape

torch.Size([768])

In [35]:
context_embedding[0]

tensor([-2.8653e-01, -9.9363e-01, -9.3477e+00, -2.5247e+00,  2.1329e-01,
         1.0827e+00, -2.8550e+00, -1.2623e+00, -3.7959e-01, -1.4560e+00,
         4.5839e-01,  1.0963e+00, -1.1531e+00, -4.2738e-01, -8.2600e-01,
        -3.1910e-01, -9.7138e-01,  2.9290e-01,  5.0975e-01,  8.6698e-01,
        -5.3529e-01, -2.1604e-01,  4.7067e-01,  2.6856e-01, -1.0440e-01,
        -3.7971e-01,  1.8070e+00,  1.1829e+00,  1.7722e+00, -2.8316e-01,
        -9.9800e-01,  1.7752e-01, -1.3530e+00, -7.0464e-01,  1.6449e+00,
         3.7872e+00, -1.0315e+00, -2.1491e-01, -7.3102e-01, -1.3870e+00,
         5.0852e-01,  5.8200e-01, -3.6066e-01,  5.9850e-01, -5.0848e-02,
         1.0676e-01,  8.9748e-01,  3.0030e-01,  5.3598e-01,  1.4670e+00,
        -9.3910e-01,  1.4268e+00, -7.5391e-01, -7.2288e-01, -1.1273e+00,
        -7.1234e-01, -2.9255e+00, -1.7096e+00,  8.0329e-01,  7.1997e-01,
        -3.4294e-01,  4.4842e-01, -2.4403e+00,  9.2369e-01, -1.1154e+00,
         3.4092e-02, -1.1507e-01,  6.7739e-01,  9.9

## A function to get the cosine distance

In [36]:
def cosine_distance(a, b):
    '''
    Calculates the cosine distance between 1-D arrays.
    '''
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return 1 - dot_product / (norm_a * norm_b)

## Check If it's working or not!

In [37]:
m = np.random.randint(0, 1000)
n = np.random.randint(0, 1000)
sim = cosine_distance(context_embedding[m], context_embedding[n])
print(f'{m}->{context_token[m]} {n} ->{context_token[n]}')
print(sim)

from scipy.spatial.distance import cosine
cos_dist = cosine(context_embedding[m], context_embedding[n])
cos_dist

566->উঠিতে 207 ->খোলা
0.4691065549850464


0.46910661458969116

## Write a function which calculates the cosine distance of a given `Word` with all other `Words` and then sort it ascendingly.

In [38]:
def list_distance(word, tokens, vectors):
    '''
    Takes a word as a parameter and find the cosine distance of this word 
    with all the other words in the token list and put the value in a list
    Then sort it in descending order and finally we return the sorted list.
    
    '''
    ans = []
    
    indices = [i for i, t in enumerate(tokens) if t == word]
    vects = [vectors[i] for i in indices]
    vec1 = vects[0]
#     print(vec1[:10])
    for i, tok in enumerate(tokens):
        temp = []
        cd = cosine_distance(vec1, vectors[i])
        temp.append(tok)
        temp.append(cd)
        ans.append(temp)
    # Sort by the second element (index 1) of each inner list
    ans.sort(key=lambda x: x[1])
    return ans


## Function to remove unwanted outputs (cleaning in multiple layers)

In [39]:
def delete_duplicates(result):
    '''
    Deletes the duplicate words from the list But keeps the smallest distance
    It also deletes some noisy(##ল্পে) Words.
    '''
    # Create an empty dictionary to store the names as keys and their 
    # corresponding smallest values as values
    name_dict = {}

    # Iterate through the data list
    for item in result:
        name, value = item[0], item[1]
        
        # Skip the item if the name contains a '#'
        if '#' in name:
            continue
            
        # If the name is not already in the dictionary or the current value 
        # is smaller than the recorded value, update the value
        if name not in name_dict or value < name_dict[name]:
            name_dict[name] = value

    # Convert the dictionary back to a list of lists
    result = [[name, value] for name, value in name_dict.items()]
    return result

In [40]:
def most_similar(positive=[],negative=[],topn=10):
    
    
    positive_vec=[]
    negative_vec=[]
    for i in positive:
        if token_df['token'].isin([i]).any():
            index=token_df.index[token_df['token'] == i].tolist()
            positive_vec.append(np.average(embedding_df['vector'][index]))
    
    for i in negative:
        if token_df['token'].isin([i]).any():
            index=token_df.index[token_df['token'] == i].tolist()
            negative_vec.append(np.average(embedding_df['vector'][index]))
            
            
    
    positive_avg=np.average(positive_vec,axis=0)
    negative_avg=-1*(np.average(negative_vec,axis=0))
    
    final_vec=(np.add(positive_avg,negative_avg))/2
    from sklearn.metrics.pairwise import cosine_similarity
    
    datadict={}
    for i in range(0,len(list1)):
        l=1
    
        datadict[i]=abs(cosine_similarity([final_vec],[list1[i]]))


    
    sorted_tuples = sorted(datadict.items(), key=lambda item: item[1])
    
    sorted_dict = {k: v for k, v in sorted_tuples}
                     
    output_index=[]
    n=0
    for i in sorted_dict:
        if n>topn:
            break
        output_index.append(i)
        n=n+1
                     
    return output_index
        

In [41]:
# list1=[]
# list2=[]
# list3=[]
# for i in range (0,len(context_embedding)):
#     list3.append(i)

# for embedings in tqdm(context_embedding):
#        list1.append(embedings.numpy())
        
# embedding_df=pd.DataFrame( {"id":list3,"vector":list1})
# embedding_df.to_csv("embedding.tsv",sep='\t',index=False,header=False)

# token_df=pd.DataFrame( {"id":list3,"token":context_token})

# token_df.to_csv("metadata.tsv",sep='\t',index=False,header=False)
# token_df.shape,embedding_df.shape,len(list3)

In [42]:
# token_df.head()

## Check the results

In [43]:
# একটা ফাংশন বানাই 
def show_similar_words(shobdo,  num=10, token = context_token, embedding = context_embedding):
    '''
    Takes a word and shows its cosine similarity with top words. Where Num represents
    number of output word we will show.
    
    Args:
        shobdo : Any Bengali Word.
        num : number of output words similar to shobdo default is 10.
    '''
    try:
        res = list_distance(shobdo, token, embedding)
        result = delete_duplicates(res)
        for val in result[:num]:
            print(f'{val[0]},', end=' ')
    except:
        print('Word is not in the context token list.')
#     return result[:num]

In [44]:
show_similar_words('প্রেম', 20)

প্রেম, মোহ, কৌশল, অস্থিরতা, মেঝে, ক্রোধ, মাছ, বন্ধু, পথ, শোক, ভবিষ্যৎ, শোনে, প্রতিনিধি, গণেশ, মালা, কৌতুক, স্ত্রী, ঝোল, মাথা, কথা, 

In [89]:
word = 'ভালবাসি'
word in context_token2, tokenizer.tokenize(word)
# list_distance(word, context_token, context_embedding)

(True, ['ভালবাসি'])

In [46]:
show_similar_words('পুরুষ', num=70)

পুরুষ, জন্ম, বাপের, পিতৃ, বৎসর, চিরদিন, ধন, স্বামী, বহুকাল, রাজ, রাত, ঋতু, আদিম, সন্তান, মাঝ, সারাদিন, মনো, জীবিকা, ঘর, দিন, কুড়ি, ভোর, আজীবন, ঝাঁক, পেট, বাঁশ, দিবার, পুরুষের, নদী, বাপ, বহু, শরৎ, সংসার, রাত্রি, স্থল, পথ, বছর, মাল, দীর্ঘ, দুঃখ, নৌকা, যাদু, ফসল, টাকার, শ্যামা, ঘরের, ক্ষণ, কাল, জেলে, চর, সারা, পুরাতন, অনেকদিন, শ্র, রাই, ভাত, রবি, পাঁচ, রাজব, কয়েকদিন, মাথা, গ্রামের, ষাট, পাড়ার, নারী, কুঁড়ে, বাড়ি, মাঝি, চৌকি, দুর্গা, 

In [47]:
show_similar_words('নারী', num=70)

নারী, মাইয়া, বৌ, বয়স্ক, মেয়ে, স্বাস্থ্য, স্ত্রীলোক, অন্ন, ধর্ম, মাছ, পাখি, যুগল, ভদ্রমহিলা, শহরের, আমিন, গ্রামের, নৌকা, ঘরের, স্ত্রীর, অভিমান, মানিক, রঙের, মুসলমান, দরিদ্র, সমুদ্র, গোর, মানব, ভদ্র, হিন্দু, চুন, ধন, আলু, রমণী, মেয়েদের, চাল, উঁচু, পুলিস, জঙ্গলের, অভিনেতা, ধানের, মানুষ, জ্বর, ছেলেমেয়ে, গোপী, মিয়ার, বুড়া, ওর, কাগ, জেলে, উপ, ক্ষুধা, গণেশ, ঠাকুর, মমতা, পোলাপান, পিস, রহস্য, অন্ধকার, গোয়াল, জীব, মাঝি, স্তন, মেজ, বিপদের, ছাগলের, গাছ, আত্মার, নিরীহ, মালা, মেয়েমানুষ, 

In [48]:
show_similar_words('প্রেম', num=70)

প্রেম, মোহ, কৌশল, অস্থিরতা, মেঝে, ক্রোধ, মাছ, বন্ধু, পথ, শোক, ভবিষ্যৎ, শোনে, প্রতিনিধি, গণেশ, মালা, কৌতুক, স্ত্রী, ঝোল, মাথা, কথা, ভাত, রমণী, রসিকতা, ভঙ্গি, মাথায়, বিবাহ, সমবেদনা, বড়লোক, যাত্রার, পূজা, ধন, জিদ, খাতির, লাভ, নৌকা, চর, বিষাদ, স্থান, কলহ, তামাক, অভিনেতা, ইতিহাস, কাহিনী, যুগল, যাত্রী, মেয়েটাকে, অলস, ঘর, রোমাঞ্চ, দেয়, ঢাকা, পাড়া, সন্তোষ, গম্ভীর, দিল, কাম, বদ, গৃহস্থ, রসুল, খাওয়া, মাঝি, বাই, গোরা, দেখে, মামা, সম্পত্তি, বৌ, যায়, কীর্তি, পোলা, 

In [49]:
show_similar_words('পদ্মা', num=70)

পদ্মা, পদ্মার, নদীর, নদীতে, আউ, নদী, চাঁদপুর, মাঠের, জলা, মাঠ, বাদল, হাওর, দাওয়া, ডোবা, সমুদ্রের, আম, পাটের, বাঁশ, ইলিশের, ঘাটে, কাশ, গোয়ালন্দ, উঠানে, বর্ষা, হাসপাতালে, জেট, রথ, গোপী, গণেশ, জেলে, কলিকাতা, কাদা, আকাশ, ময়না, হাটের, মাচ, রথের, উন, মাছ, ফরাস, শরৎ, ঘরের, ইলিশ, কুবের, মালা, রাই, নৌকার, আশ্ব, মেঝে, নৌকা, নাও, জঙ্গল, নালা, চালা, শীত, দোলে, মেলায়, তামাক, চাষী, রবি, মেলার, রাজব, মাঝি, খালে, দুর্গা, সিধু, গাছের, শ্রাবণের, বিছানা, দ্বীপ, 

In [50]:
context_token2[0], context_embedding2[0].shape

('ইছ', torch.Size([768]))

In [51]:
show_similar_words('নদী', num=100, token=context_token, embedding=context_embedding)

নদী, নদীর, পদ্মা, জেলে, ধান, পুকুর, নৌকা, জঙ্গল, পদ্মার, সমুদ্রের, নৌকার, জাহাজ, জলে, জলা, খাল, দ্বীপ, ঘাটের, আম, ঘাস, ক্ষেতের, মাছ, চাব, ঘাটে, বাঁশ, গানের, সমুদ্র, ঘরের, গ্রামের, দাওয়া, মাচ, পথের, জঙ্গলের, নৌকায়, মাছের, গাছের, উঠান, কদম, গ্রাম, পাটা, নালা, দুর্গা, বিড়ি, বর্ষা, মাল, তীর, লাউ, দ্বীপের, জলের, ঢে, মাঠ, দ্বীপে, ঘর, ভাসা, জল, ধানের, বেড়ার, হাটের, বন্দরের, পুকুরের, মাঝি, পথ, ময়না, নদীতে, মাঠে, হাসপাতালের, দুর্ভিক্ষ, বেড়া, লাঠি, মেলায়, ইলিশ, ভূমি, কাদা, জমি, বন, গণেশ, আলোক, পাটি, লঞ্চ, হাওর, বন্যার, মেঘনার, ছ, ছাউনি, পাত্র, তেঁতুল, চালা, বাড়ির, জাল, রেলস্টেশন, নাও, চর, যাত্রা, খাস, ঘটি, দরজার, মেঝে, গোয়াল, চালে, কলস, নারিকেল, 

In [52]:
show_similar_words('নদী', num=100, token=context_token2, embedding=context_embedding2)

নদী, জীব, পাখি, গাছ, শাস্ত্র, মাছ, বিচারক, প্রতিষ্ঠান, ঘোড়া, মেয়ে, দলিল, মানুষ, রাস্তা, শিশু, ব্রাহ্মণ, জাহাজ, ডাকাত, পল্লী, পোকা, কবিরাজ, মন্দির, কবর, ব্যক্তি, ঈশ্বর, বাবা, মাঠ, ডাল, নবাব, ক্ষেত, খাদ্য, পাগল, দুধ, আশ্রম, বৌ, বই, অসুখ, জমি, বোন, পুতুল, মিষ্টি, আম, লাঠি, জল, ভাত, পুরুষ, পুকুর, তত্ত্ব, শিল্প, ছায়া, লোক, জিনিস, ঘর, পুঁথি, ফুল, বাড়ি, ছেলে, পথ, গ্রাম, টাকা, সুর, গোলা, দালাল, ছবি, কাদা, গান, হাত, ইতিহাস, তেল, দোকান, বন্দুক, সাধু, চরিত্র, মালিক, শয়তান, চাঁদ, ডাক, বাঘ, খান, খায়, বাণী, সন্তান, গুরু, দেবী, বাতাস, গ্রন্থ, গন্ধ, নদীর, আকাশ, পাহাড়, ঘটনা, জীবন, সন্ন্যাসী, চুল, সাপ, শিয়াল, দেশ, চেহারা, কুকুর, দৃষ্টি, মাথা, 

আরও কিছু শব্দ

## Difference Between Two writing

In [83]:
def dif_of_book(word, limit=50):
    '''
    
    '''
    print(f"Book = {li_book[0]} Words: ")
    print(show_similar_words(word, num=limit), end='\n--------------------------\n')
    print(f"Book = {li_book[1]} Words: ")
    print(show_similar_words(word, num=limit, token=context_token2, embedding=context_embedding2))

In [74]:
dif_of_book('মাছ', 70)

Book = 1. Padma Words: 
মাছ, মাছের, ধানের, পাখি, ইলিশ, নদীর, ফসল, প্রতিমা, জল, সন্তান, পয়সা, স্নান, জলে, ধরার, বৃষ্টি, ইলিশের, শাক, ভাত, নৌকা, ধরি, জমি, তামাক, হাঁসের, ধান, নদী, মানুষ, মদ, আম, চুল, টাকা, মাল, মদের, জাহাজ, অর্থ, পোশাক, দুর্গা, গাছের, নৌকার, জুয়া, মাংস, ব্যবসা, স্রোতের, খাদ্য, সূর্য, কাপড়, চামড়া, চুড়ি, মাঝি, গাছ, কথা, দাম, শীত, জিনিসপত্র, ঢাক, অর্জন, চোখ, জাল, বিড়ি, বাতাস, রোদ, পাটা, ভাড়া, আমের, হাড়, ঘর, চারা, তেল, হাসপাতালে, কাঠের, ছেলেমেয়ে, None
--------------------------
Book = 1. Padma Words: 
মাছ, কলা, চাল, গুড়, শাড়ি, নাড়ু, চিংড়ি, বাঁশ, মশা, টাকা, কাঁঠাল, ফুল, রুই, ধান, তেল, ডাল, মাল, রস, ফল, মশাল, সাপ, জমি, লাউ, বলদ, বাগান, গাছ, তরকারি, ঘোড়া, আলু, মাংস, ভাত, দোকান, লঙ্কা, লুচি, পাহাড়, তেলের, সিঁদুর, ময়দা, গাড়ি, ধুতি, গান, চুলের, water, শাক, পায়েস, পাথর, বই, চিঠি, মাছের, পুতুল, টুকরো, ধানের, ওষুধ, বোতল, হাসি, বাচ্চা, দুধ, তামাক, ঘর, জল, মদ, গামছা, মেঘ, পয়সা, লতা, পুল, গুঁড়ি, পাখির, চুল, তাল, None


In [78]:
dif_of_book('চাষ', 100)

Book = 1. Padma Words: 
চাষ, মাঝির, গোর, ওয়া, মাঝি, জেলে, চাট, পুঁ, মাতব, চাব, কপিল, ঢে, জঙ্গল, রেল, বাম, মাচ, অভ, মাছ, ময়, চাষী, স্ট, চৌকি, ধান, নর, কয়ে, বর, রাজব, নোয়া, কাচ, ডা, নদী, আউ, গৃহ, প্রজা, আহার, চি, পালক, সিঁ, হারা, তর, কচুর, বকে, আঁশ, কুবের, নক, ঘর, বৈঠ, চোরা, তক, পাটা, পিস, কর্, খি, খাস, চ্যা, চাঁ, কুট, গ্রামের, বর্ষা, বৈ, দ্বীপে, পদ্মা, আতি, চিড়, গৃহস্থ, হোগ, ময়না, বেহ, ধানের, দেবী, দরিদ্র, ভাসা, আল, দারিদ্র্য, রাস, গাঁ, গরু, গভ, পাড়ার, ছ, কুঁড়ে, দ্বীপ, বেড়া, নি, কাছ, জামাত, হাঁটি, স্যা, বেঞ্চ, বাবু, লগ, নাতি, কমলা, দেনা, পাট, বুনো, ছেলেমে, গরীব, কুটির, রাজ, None
--------------------------
Book = 1. Padma Words: 
চাষ, পাড়া, কুম, মুচ, খান, সতী, চণ্ড, চাল, তপ, ইছ, গোর, টম, বাসন, লাঠি, গার, কলক, নার, বাম, কুল, ধান, ঝাড়, ঢে, কলম, সাপ, জগ, গোয়াল, রাম, পল্লী, সাধু, সো, দুর্গাপ, ট, পেট, চি, শাম, মাতব, ধা, মাদ, আম, অন্ন, খে, গ্রাম্য, কাঠ, চাকর, রাঁধ, কৃ, তর, গা, জীবন, নাপ, দর, বেত, ব্যবসা, নীল, আখ, কৃষিক, আড়, কচুর, রাশ, ভিক্ষ, কুলি, আমল, আমোদ, সরা, চাষী, ডুম, কুঠ, উ

In [69]:
word = 'বিদ্রোহ'
limit = 70
print(f"Book = {li_book[0]} Words: ")
print(show_similar_words(word, num=limit), end='\n--------------------------\n')
print(f"Book = {li_book[0]} Words: ")
print(show_similar_words(word, num=limit, token=context_token2, embedding=context_embedding2))

Book = 1. Padma Words: 
Word is not in the context token list.
None
--------------------------
Book = 1. Padma Words: 
বিদ্রোহ, সন্ন্যাস, লজ্জা, বসন্ত, রসিক, দেওয়ান, নিস্তার, অমৃত, গুণ, শক্তি, বুনো, সাধন, ফাঁসি, সাহসিক, গোরা, সন্ন্যাসী, সন্ধ্যা, সাধু, নন্দ, রাজার, ভক্তি, মায়া, বিকার, কবিরাজ, সাহসী, আদর, পণ্ডিত, অত্যাচার, রথ, তর্ক, চমক, গ্রাম্য, মন্ত্র, রত্ন, ঠাকুর, জেলে, ভাগ, রোগ, সিদ্ধ, চৈতন্য, আদি, তপ, নারায়ণ, মুক্ত, ব্রাহ্মণ, ঘুম, দুঃখ, সন্ন, নায়েব, দুর্দান্ত, বাহাদুর, সুবাস, যাত্রার, দেব, বিষয়, রস, যুগ, লাঠি, আস্থা, বধূ, সমাজ, শিল্প, ভীতি, ব্রহ্ম, মিনতি, আমিন, নবাব, বন্য, কলা, বিনয়, None


In [79]:
dif_of_book('নারী', 100)

Book = 1. Padma Words: 
নারী, মাইয়া, বৌ, বয়স্ক, মেয়ে, স্বাস্থ্য, স্ত্রীলোক, অন্ন, ধর্ম, মাছ, পাখি, যুগল, ভদ্রমহিলা, শহরের, আমিন, গ্রামের, নৌকা, ঘরের, স্ত্রীর, অভিমান, মানিক, রঙের, মুসলমান, দরিদ্র, সমুদ্র, গোর, মানব, ভদ্র, হিন্দু, চুন, ধন, আলু, রমণী, মেয়েদের, চাল, উঁচু, পুলিস, জঙ্গলের, অভিনেতা, ধানের, মানুষ, জ্বর, ছেলেমেয়ে, গোপী, মিয়ার, বুড়া, ওর, কাগ, জেলে, উপ, ক্ষুধা, গণেশ, ঠাকুর, মমতা, পোলাপান, পিস, রহস্য, অন্ধকার, গোয়াল, জীব, মাঝি, স্তন, মেজ, বিপদের, ছাগলের, গাছ, আত্মার, নিরীহ, মালা, মেয়েমানুষ, বাচ্চা, পুরুষের, মেয়ের, কুব, ঘুমের, কালা, পরিবার, শীর্ণ, নৌকার, কালো, ছাগল, হোসেনের, রাগ, স্ত্রী, দিদির, বাণিজ্য, গ্রাম, লীলা, ঘর, মুসলমানের, উলঙ্গ, বিদ, হাঁটু, বিবাহের, পঙ্গু, কুক, পায়ের, মেয়েকে, মুড়ি, পুরুষ, None
--------------------------
Book = 1. Padma Words: 
নারী, মেয়ে, স্ত্রীলোক, বধূ, দেবী, বাঁশ, বৌ, মেম, ঠাকুরের, তরুণীর, মানুষ, শ্যামা, অল্প, মেয়েটি, শাস্ত্র, শিমুল, ফুল, জলের, মেয়েরা, বট, বৌদি, শাড়ি, সন্ন্যাসী, আম, দেবীর, পুত্র, বিধবা, রাঁধ, জীব, বন্য, রেল, পুষ্প, জল, 

In [81]:
dif_of_book('বিদেশ')

Book = 1. Padma Words: 
বিদেশ, বিদেশে, হাসপাতাল, শহরে, মেলায়, ঢাকায়, হাসপাতালে, চাঁদপুর, দ্বীপে, জাহাজে, কোথাও, দূরে, নৌকায়, আড়ালে, কলিকাতা, হাটে, বাড়ি, ঘুম, জেল, এখানে, অজানা, মালদহ, বাহিরে, উধাও, উঠানে, চলিয়া, নিরুদ্দেশ, নগর, উজান, গাঁয়ে, বাহির, গ্রামে, মাইয়া, দূর, নরকে, নীচে, শ্বশুরবাড়ি, পদ্মা, জেলে, রাজ্য, চুরি, মাছ, গেলি, চাল, ঘাটে, নদীতে, নৌকা, তামাক, ওখানে, পদ্মার, None
--------------------------
Book = 1. Padma Words: 
বিদেশ, স্বর্গ, সাধু, সাহেব, যৌবন, গরিব, রেল, বাবু, নক্ষত্র, বন, বিশ্ব, মেম, ধোঁয়া, টাকা, ডাক্তার, নেশা, কলকাতার, কলকাতা, চৈতন্য, ন্যায়, সমুদ্র, ছেলে, বর্ষা, ভগবান, ব্রাহ্মণ, তীর্থ, ঘুষ, বুড়ো, দাঙ্গা, অমৃত, বর্ধমান, পাথর, বিত্ত, লুচি, শিল্প, বিলাস, জগৎ, কুঞ্জ, পুরুষ, ব্যবসা, পাপ, সন্ন্যাসী, দূর, ভুবন, তিলক, শখ, আসবাব, বেগুন, কলকাতায়, জোয়ান, None


In [85]:
dif_of_book('সমাজ', 100)

Book = 1. Padma Words: 
সমাজ, দল, পাল, জীবন, পাড়া, জায়গা, গ্রাম, আত্মীয়, সংসার, মানুষ, লোক, পরিবার, পথ, সম্পত্তি, প্রতিনিধি, ঘর, ব্যবসা, জীবিকা, যুবক, ভাব, তাগিদ, দ্বীপ, পৃথিবী, পোলাপান, বাসিন্দা, সঙ্গত, সমষ্টি, সমাজে, বিছানা, গৃহস্থ, মন, মা, বাণিজ্য, পুরুষ, ছেলে, মুসলমান, প্রেম, ঝড়, দ্বীপে, আবহাওয়া, বৌ, মালা, কুমারী, বাড়ির, পরিবর্তন, অসুখ, মিয়া, ভাষা, রাজা, মাঝি, প্রাণ, মাখন, বাই, প্রক্রিয়া, পোলা, গা, কুয়াশা, কাণ্ড, বোঝাই, বাপ, কৌতুক, উপায়, দেশ, বাড়িতে, দাম, জগতে, ইতিহাস, হাট, ফাঁকি, পরিপূর্ণ, মেয়েটা, বোন, শীতকাল, সকলে, বিষাদ, পরিচালক, রূপ, আবৃত, হোটেল, মালিক, অঙ্গ, বন, রাজ্য, বুক, প্রকৃতি, ছল, ছেলেমানুষ, পারা, চিহ্ন, ব্যবহার, বুদ্ধি, দাস, কাহিনী, বাবু, কর্তা, শিকারী, সিং, আড্ডা, মেয়েকে, বন্ধু, None
--------------------------
Book = 3. Isamoti Words: 
সমাজ, প্রতিবেশ, গ্রাম্য, কুল, নীল, সংসার, জীবন, সরকার, ঠাকুর, লাঠি, বাঁশ, ব্রহ্ম, ভক্তি, তপ, রাজ, মোড়ল, সাধু, গ্রামের, গাঁয়ের, রেল, গঙ্গা, আড়ত, মন, সমাজের, ব্রাহ্মণ, সংসারের, রাম, পাড়ার, দল, সতী, ঈশ্বর, দেওয়ান, জাহাজ, প

In [90]:
dif_of_book('ভালবাসি')

Book = 1. Padma Words: 
ভালবাসি, তাকাই, ঘুমানোর, বুঝি, ধরি, স্নেহ, কাঁদি, চিনি, খাই, চিন্তা, শুনি, চাহ, বন্ধুত্ব, স্বার্থপর, খাওয়া, ভালবাসে, কিনি, রাঁধ, মুগ্ধ, মারি, ভাবি, কাটাই, ফেলি, খাতির, কান্দ, বাঁচি, ঘুমাই, চালাই, ঠক, অপমান, সাজ, হাঁটি, স্নান, থাকি, পালানোর, কাঁদ, চিন, রাগ, জমা, সেবা, মরি, ব্যাকুল, পছন্দ, বসি, কাটি, সুখের, আদর, ভালবাসা, দুঃখ, করিবার, None
--------------------------
Book = 3. Isamoti Words: 
ভালবাসি, ভালোবাসি, ভালবাস, ভালবাসে, ভালবাসতে, শ্রদ্ধা, চেনে, পাই, শেখা, ভালোবাসেন, পছন্দ, সুখী, ভালোবাসে, পড়া, মানি, দরকার, ভালো, চাই, শিখ, খিদে, গরিব, লাগে, রাঁধ, রাখি, মুগ্ধ, গাই, কেঁদে, খাই, আপনজন, করেন, নেই, অনুরাগ, সুখ, থাকি, চিন, বোস, খা, চিনতে, সুন্দর, ঈর্ষা, অজ্ঞ, চায়, উপাসনা, বক, জ্ঞানী, করেছি, ভাবেন, জানি, মরে, খুশি, None


In [ ]:
import torch

In [ ]:
x = torch.Tensor([
    [1,2],
    [1,2],
    [1,2]
])
x[:,1]

In [ ]:
show_similar_words('গুরু', 30)

In [ ]:
show_similar_words('ঈশ্বর', 30)

In [ ]:
show_similar_words('ডাক্তার', 30)

In [ ]:
import os

filepath = os.path.join('/kaggle/working/')

In [ ]:
name = 'metadata.tsv'
with open(os.path.join(filepath, name), 'w+') as file_metadata:
    for i, token in enumerate(context_token):
        file_metadata.write(token + '\n')

In [ ]:
import csv

name = 'embeddings.tsv'

with open(os.path.join(filepath, name), 'w+') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    for embedding in context_embedding:
        writer.writerow(embedding.numpy())